In [1]:
"""
Check all datasets
"""
%matplotlib inline
%load_ext autotime
%load_ext autoreload
%autoreload
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-deep')
from ipywidgets import interact
from ipywidgets import Controller

from collections import defaultdict
import numpy as np
import os
import sys
import itertools
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from pprint import pprint
import regex as re
sys.path.append('/export/home/asanakoy/workspace/neural_network')
sys.path.append('/export/home/asanakoy/workspace/art_datasets')
sys.path.append('/export/home/asanakoy/workspace/dataset_merger')
import make_data.dataset
import wikiart.info.preprocess_info
from art_utils.pandas_tools import is_null_object
from art_utils.text_tools import extract_all_years
import dataset_merger.read_datasets
import dataset_merger.pymongoext as pymongoext
from dataset_merger.match_artists import fix_sim_matrix
from dataset_merger.match_artists import generate_matches_for_manual_check
from dataset_merger.match_artists import get_num_top_matches
from dataset_merger.match_artists import compute_sim_matrix, get_sim_between_rows
from dataset_merger.match_artists import find_connected_components
from dataset_merger.match_artists import combine_artists
from dataset_merger.match_artists import get_merged_artists_df
import dataset_merger.prepare_artists as prepare_artists
from dataset_merger.prepare_artists import get_artists_with_years

pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 160)
pd.set_option('display.max_colwidth', 50)  
pd.set_option('display.max_columns', None) 

# todo: merge wga, wikiart, artuk, googleart (dates checked)
# todo: make uniform dates (bio or active if no bio available) range for every artist in the dataset

# todo: check dates on others
# todo: chach should we split naems on `&` for other datasets (beyound artuk, wikiart, googleart, wga)
# TOD: manually check connected components (wga, artuk)

In [2]:
dfs = dataset_merger.read_datasets.read_datasets()

artuk None
artuk None 207585
googleart None
googleart None 102343
moma None
moma None 24708
rijks None
rijks None 93206
wga None
wga None 35373
wiki None
wiki None 132296
meisterwerke None
meisterwerke None 19647
Total works count: 615158
time: 24.4 s


In [3]:
# for key, df in dfs.iteritems():
#     print key
#     print df.columns
#     print '==='
    

# print '--'
# artist_names = np.unique(artist_names)
# print 'Total unique artists num:', len(artist_names)
# return artist_names

time: 2.33 ms


In [4]:
artists_with_years_dict = dict()
artists_with_years_dict['googleart'] = get_artists_with_years('googleart', dfs)
artists_with_years_dict['wiki'] = get_artists_with_years('wiki', dfs)
artists_with_years_dict['wga'] = get_artists_with_years('wga', dfs)
artists_with_years_dict['artuk'] = get_artists_with_years('artuk', dfs)
artists_with_years_dict['meisterwerke'] = get_artists_with_years('meisterwerke', dfs)
artists_with_years_dict['moma'] = get_artists_with_years('moma', dfs)

:: googleart


 32%|███▏      | 1355/4222 [00:00<00:00, 13543.23it/s]

with url_wiki: 3074


100%|██████████| 4222/4222 [00:00<00:00, 14922.64it/s]


Total artists: 4222
Drop artists with NaN years_range 138
Drop 1 unknown artists
Total artists: 4083
:: wiki


...


Total artists: 2218
:: moma


 12%|█▏        | 1787/15247 [00:00<00:00, 17861.80it/s]

with url_wiki: 0


100%|██████████| 15247/15247 [00:00<00:00, 17329.31it/s]


Total artists: 15247
Drop artists with NaN years_range 3576
Drop 1 unknown artists
Total artists: 11670
time: 16.2 s


In [5]:
# artcyclopedia_artists = pymongoext.get_db_entries('129.206.117.36', 27017, 'artcyclopedia', 'found')
# for x in artcyclopedia_artists:
#     x['artcylopedia_name'] = x['name']
#     del x['name']
#     del x['_id']
#     if 'content' in x:
#         del x['content']
# print len(artcyclopedia_artists)
# print artcyclopedia_artists[:2]

# artcyclopedia_df = pd.DataFrame.from_dict(artcyclopedia_artists)
# artcyclopedia_df.index = artcyclopedia_df['artist_name']
# artcyclopedia_df.rename(columns={'url': 'url_artcyclopedia'}, inplace=True)

# # artists_df['url_artcyclopedia'] = artcyclopedia_df.loc[artist_names, 'url_artcyclopedia'].values
# # print pd.notnull(artists_df['url_artcyclopedia']).sum()
# print len(artcyclopedia_df)
# print len(artcyclopedia_df['url_artcyclopedia'].unique())


# artcyclopedia_df

time: 4.6 ms


In [6]:
def get_sim_matrix(dataset_names, shape=None, force=False, n_jobs=1, num_blocks=None):
    assert len(dataset_names) == 2
    str_name = '-'.join(dataset_names)
    matrix_filepath = 'sim_matrix_{}.npy'.format(str_name)
    if os.path.exists(matrix_filepath) and not force:
        sim = np.load(matrix_filepath)
        if shape is not None:
            assert shape == sim.shape, '{} != {}'.format(shape, sim.shape)
    else:
        sim = compute_sim_matrix(keys, artists_with_years_dict, n_jobs=n_jobs, num_blocks=num_blocks)
        np.save(matrix_filepath, sim)
        print 'Sim matrix saved to {}'.format(matrix_filepath)
    return sim

def get_sim_matrix_and_plot(dataset_names, force=False, n_jobs=1, num_blocks=None):
    str_name = '-'.join(dataset_names)
    print str_name
    shape = tuple([len(artists_with_years_dict[dataset_names[i]]) for i in xrange(2)])
    sim_matrix[str_name] = get_sim_matrix(dataset_names, shape=shape, force=force, 
                                          n_jobs=n_jobs, num_blocks=num_blocks)
    print sim_matrix[str_name].shape
    best_sim = sim_matrix[str_name].max(axis=1)
    plt.hist(best_sim)
    plt.show()

sim_matrix = dict()

time: 20.7 ms


In [7]:
def check_pairs(dataset_names, transpose=False, 
                min_sim=85, max_sim=100, 
                discard_exaclty_matched_dates=False,
                min_k=1, min_total_works_count=0):
    assert len(dataset_names) == 2
    sim = sim_matrix['-'.join(dataset_names)]
    dfs_to_merge = [artists_with_years_dict[key] for key in dataset_names]

    manually_checked_matches_path = 'manually_corrected_matches/{}_manually_corrected_matches.csv'.format('-'.join(dataset_names))
    print manually_checked_matches_path
    if os.path.exists(manually_checked_matches_path):
        manually_checked_matches_df = pd.read_csv(manually_checked_matches_path, index_col=0, encoding='utf-8')
        print 'manually checked pairs:', len(manually_checked_matches_df)
        sim = fix_sim_matrix(dataset_names, 
                             dfs_to_merge, 
                             sim, 
                             manually_checked_matches_df)
    
    if transpose:
        # Transpose matches
        dataset_names = dataset_names[::-1]
        dfs_to_merge = dfs_to_merge[::-1]
        sim = sim.transpose()

    results_df = generate_matches_for_manual_check(dataset_names, dfs_to_merge, 
                                                   sim, min_sim=min_sim, max_sim=max_sim,
                                                   discard_exaclty_matched_dates=discard_exaclty_matched_dates,
                                                   min_k=min_k, 
                                                   min_total_works_count=min_total_works_count)
    output_path = '/export/home/asanakoy/workspace/dataset_merger/aggregated/{}_check_matches.csv'.format('-'.join(dataset_names))
    results_df.to_csv(output_path, encoding='utf-8')
    return results_df

time: 25.6 ms


In [8]:
def correct_ids_in_manually_checked_matches(dataset_names):
    """
        Check each pair in manually_checked_matches and change $artist_id to one from 'artist_ids' list (if possible)
        if $artist_id is not currently in index of the artist_df
    """
    
    manually_checked_matches_path = 'manually_corrected_matches/{}_manually_corrected_matches.csv'.format('-'.join(dataset_names))
    if not os.path.exists(manually_checked_matches_path):
        raise IOError('File {} not found'.format(manually_checked_matches_path))
    manually_checked_matches_df = pd.read_csv(manually_checked_matches_path, index_col=0, encoding='utf-8')
    print 'manually checked pairs:', len(manually_checked_matches_df)

    dfs_to_merge = [artists_with_years_dict[key] for key in dataset_names]
    
    def convert_to_valid_idx(artists_df, artist_id):
        if artist_id not in artists_df.index:
            found_idxs = np.nonzero(artists_df['artist_ids'].apply(lambda x: artist_id in x))[0]
            if len(found_idxs):
                assert len(found_idxs) == 1, '{}: {}'.format(artist_id, found_idxs)
                artist_id = artists_df.index[found_idxs[0]]
        return artist_id
        
    
    def is_valid_pairs(row):
        for i in xrange(2):
            artist_id = row['artist_id_' + dataset_names[i]]
            if artist_id not in dfs_to_merge[i].index:
                return False
        return True
    
    is_valid = manually_checked_matches_df.apply(is_valid_pairs, axis=1)
    print 'not valid before fixes:', (~is_valid).sum()
    print manually_checked_matches_df[~is_valid]
    
    for i in xrange(2):
        manually_checked_matches_df['artist_id_' + dataset_names[i]] = \
            manually_checked_matches_df['artist_id_' + dataset_names[i]] \
            .apply(lambda x: convert_to_valid_idx(dfs_to_merge[i], x))
    
    is_valid = manually_checked_matches_df.apply(is_valid_pairs, axis=1)
    print 'not valid', (~is_valid).sum()
    print manually_checked_matches_df[~is_valid]
    return manually_checked_matches_df

# dataset_names = ['wiki+googleart', 'wga']
# dataset_names = ['wiki+googleart+wga', 'meisterwerke']
# manually_checked_matches_df = correct_ids_in_manually_checked_matches(dataset_names)
# manually_checked_matches_path = 'manually_corrected_matches/{}_manually_corrected_matches.csv'.format('-'.join(dataset_names))
# manually_checked_matches_df.to_csv(manually_checked_matches_path, encoding='utf-8')

time: 43.2 ms


In [9]:
# def fix_wga_ids(matches_df):
    
#     def years_range_to_suffix(years_range):
#         if not isinstance(years_range, float):
#             assert isinstance(years_range, list)
#             return ', ({}-{})'.format(*np.asarray(years_range, dtype=int))
#         else:
#             return ''
    
#     matches_df = matches_df.copy()
#     fixed_bio_df = pd.read_csv('/export/home/asanakoy/workspace/wga/info/duplicate_artists_fixed_bio.csv', encoding='utf-8', header=0)
#     for row in fixed_bio_df.itertuples():
#         new_years_bio = map(int, re.split(r',|-', row.fixed_bio.strip()))
#         mask = matches_df['artist_id_wga'] == row.artist_id
#         matches_df.loc[mask, 'dates_wga'] = matches_df.loc[mask, 'dates_wga'].apply(lambda _: new_years_bio)
#         matches_df.loc[mask, 'artist_id_wga'] = matches_df.loc[mask, 'artist_id_wga'].apply(lambda x: x.rsplit(',', 1)[0] + 
#                                                                            years_range_to_suffix(new_years_bio))
#     return matches_df

# dataset_names = ['wiki+googleart', 'wga']
# manually_checked_matches_df = fix_wga_ids(manually_checked_matches_df)
# manually_checked_matches_path = 'manually_corrected_matches/{}_manually_corrected_matches.csv'.format('-'.join(dataset_names))
# manually_checked_matches_df.to_csv(manually_checked_matches_path, encoding='utf-8')

time: 4.78 ms


In [ ]:
%autoreload

time: 26.3 ms


In [ ]:
keys = ['wiki', 'googleart']
get_sim_matrix_and_plot(keys, force=True, n_jobs=16, num_blocks=512)

# keys = ['wga', 'artuk']
# get_sim_matrix_and_plot(keys)

wiki-googleart


  0%|          | 0/512 [00:00<?, ?it/s]

Num tasks: 512


 18%|█▊        | 93/512 [00:37<04:36,  1.51it/s]

In [ ]:
%autoreload
dataset_names = ['wiki', 'googleart']
results_df = check_pairs(dataset_names, 
                transpose=False,
                min_sim=76, max_sim=101, 
                discard_exaclty_matched_dates=True,
                min_k=1)

if len(results_df):
    results_df['year_diff'] = results_df.apply(lambda x: [abs(x['dates_' + dataset_names[0]][i] - x['dates_' + dataset_names[1]][i]) for i in range(2)], axis=1)
results_df

# Transposed
results_df_transposed = check_pairs(dataset_names, 
                transpose=True,
                min_sim=76, max_sim=101, 
                discard_exaclty_matched_dates=True,
                min_k=1)
if len(results_df_transposed):
    results_df_transposed['year_diff'] = results_df_transposed.apply(lambda x: [abs(x['dates_' + dataset_names[0]][i] - x['dates_' + dataset_names[1]][i]) for i in range(2)], axis=1)

if len(results_df):
    appended = results_df.append(results_df_transposed, ignore_index=True)
    appended = appended.drop_duplicates(['artist_id_' + key for key in dataset_names])
    appended = appended[results_df.columns]
else:
    appended = results_df_transposed

# appended = results_df
output_path = '/export/home/asanakoy/workspace/dataset_merger/aggregated/{}_check_matches.csv'.format('-'.join(dataset_names))
appended.to_csv(output_path, encoding='utf-8')
print 'Total rows', len(appended)
appended

In [ ]:
def check_sim_values(dataset_names, artist_ids):
    cur_dfs = [artists_with_years_dict[key] for key in dataset_names]
    indices = [0, 0]
    for i in xrange(2):
        idx = np.nonzero(cur_dfs[i]['artist_id'] == artist_ids[i])[0][0]
        print cur_dfs[i].iloc[idx]
        print '---'
        indices[i] = idx
    print indices
    sim = get_sim_matrix(dataset_names, force=False)
    print 'sim=', sim[indices[0], indices[1]]

check_sim_values(['wiki', 'googleart'], ['wiki_johan-hendrik-weissenbruch', 'googleart_m026t8y5'])

In [ ]:
dataset_names = ['wiki', 'googleart']
sim = sim_matrix['-'.join(dataset_names)]
dfs_to_merge = [artists_with_years_dict[key] for key in dataset_names]

merged_df = get_merged_artists_df(dataset_names, dfs_to_merge, sim, allow_big_components=False)
artists_with_years_dict['wiki+googleart'] = merged_df

In [ ]:
%autoreload
keys = ['wiki+googleart', 'wga']
get_sim_matrix_and_plot(keys, force=True, n_jobs=16)

In [ ]:
%autoreload
dataset_names = ['wiki+googleart', 'wga']
results_df = check_pairs(dataset_names, 
                transpose=False,
                min_sim=80, max_sim=121, 
                discard_exaclty_matched_dates=False,
                min_k=2)

if len(results_df):
    results_df['year_diff'] = results_df.apply(lambda x: [abs(x['dates_' + dataset_names[0]][i] - x['dates_' + dataset_names[1]][i]) for i in range(2)], axis=1)
results_df

# Transposed
results_df_transposed = check_pairs(dataset_names, 
                transpose=True,
                min_sim=80, max_sim=121, 
                discard_exaclty_matched_dates=False,
                min_k=2)
if len(results_df_transposed):
    results_df_transposed['year_diff'] = results_df_transposed.apply(lambda x: [abs(x['dates_' + dataset_names[0]][i] - x['dates_' + dataset_names[1]][i]) for i in range(2)], axis=1)

if len(results_df):
    appended = results_df.append(results_df_transposed, ignore_index=True)
    appended = appended.drop_duplicates(['artist_id_wga', 'artist_id_wiki+googleart'])
    appended = appended[results_df.columns]
else:
    appended = results_df_transposed

# appended = results_df
output_path = '/export/home/asanakoy/workspace/dataset_merger/aggregated/{}_check_matches.csv'.format('-'.join(dataset_names))
appended.to_csv(output_path, encoding='utf-8')
print 'Total rows', len(appended)
appended

In [ ]:
# artists_with_years_dict['googleart']['url_wiki'] = artists_with_years_dict['googleart']['url_wiki'].apply(lambda x: 'https://en.wikipedia.org/wiki/ernest_meissonier' if x == 'http://en.wikipedia.org/wiki/jean-louis-ernest_meissonier' else x)


# ids = list()
# for comp in big_comp_wiki_googleart:
#     comp = sorted(list(comp))http://localhost:8893/notebooks/dataset_merger/aggregated/match_artists.ipynb#
#     ids.extend(comp[:2])
# artists_with_years_dict['wiki+googleart'][['artist_name', 'years_range', 'url_wiki']].iloc[ids]

In [ ]:
dataset_names = ['wiki+googleart', 'wga']
sim = sim_matrix['-'.join(dataset_names)]
dfs_to_merge = [artists_with_years_dict[key] for key in dataset_names]

merged_df = get_merged_artists_df(dataset_names, dfs_to_merge, sim, allow_big_components=False)
artists_with_years_dict['+'.join(dataset_names)] = merged_df
merged_df.to_hdf('artists_{}.hdf5'.format('+'.join(dataset_names)), 'df', mode='w')
# merged_df

In [ ]:
artists_with_years_dict.keys()

In [ ]:
dataset_names = ['wiki', 'googleart', 'wga', 'wiki+googleart+wga']
for key in dataset_names:
    print '{}: {} unique artists'.format(key, len(artists_with_years_dict[key]))

In [ ]:
##### MEISTERWERKE #######

%autoreload
keys = ['wiki+googleart+wga', 'meisterwerke']
get_sim_matrix_and_plot(keys, force=True, n_jobs=16)

In [ ]:
def get_pairs_from_the_same_dataset(dataset_names, df, min_sim=102):
    df = df[df['score'] >= min_sim].copy()
    fir_ids_col = 'artist_id_' + dataset_names[0]
    sec_ids_col = 'artist_id_' + dataset_names[1]
    df.index = df[sec_ids_col]
    df_gr = pd.DataFrame(data={sec_ids_col: 
                            df.groupby(fir_ids_col)[sec_ids_col].apply(list)})
    df_gr['dates'] = df_gr[sec_ids_col].apply(lambda x: [df.at[el, 'dates_' + dataset_names[1]] for el in x])
    df_gr[sec_ids_col] = df_gr[sec_ids_col].apply(lambda x: ';'.join(x))
    return df_gr

In [ ]:
##### MEISTERWERKE #######
%autoreload
dataset_names = ['wiki+googleart+wga', 'meisterwerke']
results_df = []
results_df = check_pairs(dataset_names, 
                transpose=False,
                min_sim=85, max_sim=101, 
                discard_exaclty_matched_dates=True,
                min_k=1,
                min_total_works_count=1)

if len(results_df):
    results_df['year_diff'] = results_df.apply(lambda x: [abs(x['dates_' + dataset_names[0]][i] - x['dates_' + dataset_names[1]][i]) for i in range(2)], axis=1)
results_df

# Transposed
results_df_transposed = []
results_df_transposed = check_pairs(dataset_names, 
                transpose=True,
                min_sim=85, max_sim=101, 
                discard_exaclty_matched_dates=True,
                min_k=1,
                min_total_works_count=1)
if len(results_df_transposed):
    results_df_transposed['year_diff'] = results_df_transposed.apply(lambda x: [abs(x['dates_' + dataset_names[0]][i] - x['dates_' + dataset_names[1]][i]) for i in range(2)], axis=1)

if len(results_df) and len(results_df_transposed):
    appended = results_df.append(results_df_transposed, ignore_index=True)
    appended = appended.drop_duplicates(['artist_id_' + key for key in dataset_names])
    appended = appended[results_df.columns]
#     assert len(appended['artist_id_meisterwerke'].unique()) == len(appended), len(appended['artist_id_meisterwerke'].unique())
    appended.sort_values(by='artist_id_wiki+googleart+wga', inplace=True)
elif len(results_df):
    appended = results_df
elif len(results_df_transposed):
    appended = results_df_transposed
else:
    appended = pd.DataFrame()
    
# appended.sort_values(by='artist_id_wiki+googleart+wga', inplace=True)
# appended.sort_values(by='artist_id_meisterwerke', inplace=True)


from os.path import expanduser
output_path = expanduser('~/workspace/dataset_merger/aggregated/{}_check_matches.csv'.format('-'.join(dataset_names)))
appended.to_csv(output_path, encoding='utf-8')
print 'Total rows', len(appended)
print dataset_names
appended
# duplicates_df = get_pairs_from_the_same_dataset(dataset_names[::-1], appended, min_sim=100)
# output_path = expanduser('~/workspace/dataset_merger/aggregated/{}_duplicates.csv'.format(dataset_names[1]))
# duplicates_df.to_csv(output_path, encoding='utf-8')
# duplicates_df

In [ ]:
# results_df = appended1
# appended = results_df.append(results_df_transposed, ignore_index=True)
# appended = appended.drop_duplicates(['artist_id_' + key for key in dataset_names])
# appended = appended[results_df.columns]
# #     assert len(appended['artist_id_meisterwerke'].unique()) == len(appended), len(appended['artist_id_meisterwerke'].unique())
# appended.sort_values(by='artist_id_wiki+googleart+wga', inplace=True)
# appended.to_csv(output_path, encoding='utf-8')
# appended


In [ ]:
##### MEISTERWERKE #######
dataset_names = ['wiki+googleart+wga', 'meisterwerke']
sim = sim_matrix['-'.join(dataset_names)]
dfs_to_merge = [artists_with_years_dict[key] for key in dataset_names]

# artists_with_years_dict['+'.join(dataset_names)] = get_merged_artists_df(dataset_names, dfs_to_merge, sim)


assert len(dataset_names) == len(dfs_to_merge)
assert sim.shape == (len(dfs_to_merge[0]), len(dfs_to_merge[1])), sim.shape


manually_checked_matches_path = 'manually_corrected_matches/{}_manually_corrected_matches.csv'.format(
    '-'.join(dataset_names))
if os.path.exists(manually_checked_matches_path):
    manually_checked_matches_df = pd.read_csv(manually_checked_matches_path, index_col=0,
                                              encoding='utf-8')
    sim = fix_sim_matrix(dataset_names,
                                dfs_to_merge,
                                sim,
                                manually_checked_matches_df)

connected_components = find_connected_components(sim, min_sim=100)
big_comp = [x for x in connected_components if len(x) > 2]
small_comp = [x for x in connected_components if len(x) <= 2]
# connected_components = small_comp + [{el} for x in big_comp for el in x]
# big_comp = [x for x in connected_components if len(x) > 2]

print 'num big_comp', len(big_comp)
# assert len(big_comp) == 0, len(big_comp)
big_comp

In [ ]:
# TODO: filter out big components. Recheck them
# TODO: merge images wga wiki. googleart
# these are mostly followers of the artists in the meisterwerke and workshops.
# But they have to be manually checked
big_comp
items_list = []
for comp in big_comp:
    for item_id in comp:
        df_idx = item_id / int(1e6)
#         if df_idx == 0:
#             continue
        artist_idx = item_id % int(1e6)
        items_list.append(dfs_to_merge[df_idx].iloc[artist_idx][['artist_id', 'artist_names', 'years_range']])
pd.DataFrame(items_list)

In [ ]:
%autoreload
dataset_names = ['wiki+googleart+wga', 'meisterwerke']
sim = sim_matrix['-'.join(dataset_names)]
dfs_to_merge = [artists_with_years_dict[key] for key in dataset_names]

merged_df = get_merged_artists_df(dataset_names, dfs_to_merge, sim, 
                                  split_big_components=False, 
                                  allow_big_components=False)
artists_with_years_dict['+'.join(dataset_names)] = merged_df
print 'Num unique artists:', len(merged_df)
merged_df

In [ ]:
dataset_names = ['wiki', 'googleart', 'wga', 'meisterwerke', 'moma', 'wiki+googleart+wga+meisterwerke']
for key in dataset_names:
    print '{}: {} unique artists'.format(key, len(artists_with_years_dict[key]))

In [ ]:
##### MOMA #######

%autoreload
keys = ['wiki+googleart+wga+meisterwerke', 'moma']
mask = (artists_with_years_dict['moma']['wikidata_qid'].notnull() | 
        artists_with_years_dict['moma']['years_bio'].notnull() |
        artists_with_years_dict['moma']['works_count'] > 0)
artists_with_years_dict['moma'] = artists_with_years_dict['moma'][mask].copy()
get_sim_matrix_and_plot(keys, force=True, n_jobs=40, num_blocks=160)

In [ ]:
##### MOMA #######
%autoreload
dataset_names = ['wiki+googleart+wga+meisterwerke', 'moma']
results_df = []
results_df = check_pairs(dataset_names, 
                transpose=False,
                min_sim=84, max_sim=101, 
                discard_exaclty_matched_dates=True,
                min_k=1,
                min_total_works_count=0)

if len(results_df):
    results_df['year_diff'] = results_df.apply(lambda x: [abs(x['dates_' + dataset_names[0]][i] - x['dates_' + dataset_names[1]][i]) for i in range(2)], axis=1)
results_df

# Transposed
results_df_transposed = []
results_df_transposed = check_pairs(dataset_names, 
                transpose=True,
                min_sim=84, max_sim=101, 
                discard_exaclty_matched_dates=True,
                min_k=1,
                min_total_works_count=0)
if len(results_df_transposed):
    results_df_transposed['year_diff'] = results_df_transposed.apply(lambda x: [abs(x['dates_' + dataset_names[0]][i] - x['dates_' + dataset_names[1]][i]) for i in range(2)], axis=1)

if len(results_df) and len(results_df_transposed):
    appended = results_df.append(results_df_transposed, ignore_index=True)
    appended = appended.drop_duplicates(['artist_id_' + key for key in dataset_names])
    appended = appended[results_df.columns]
#     assert len(appended['artist_id_meisterwerke'].unique()) == len(appended), len(appended['artist_id_meisterwerke'].unique())
    appended.sort_values(by='artist_id_' + dataset_names[0], inplace=True)
elif len(results_df):
    appended = results_df
elif len(results_df_transposed):
    appended = results_df_transposed
else:
    appended = pd.DataFrame()
    
# appended.sort_values(by='artist_id_wiki+googleart+wga', inplace=True)
# appended.sort_values(by='artist_id_meisterwerke', inplace=True)


from os.path import expanduser
output_path = expanduser('~/workspace/dataset_merger/aggregated/{}_check_matches.csv'.format('-'.join(dataset_names)))
appended.to_csv(output_path, encoding='utf-8')
print 'Total rows', len(appended)
print dataset_names
appended
# duplicates_df = get_pairs_from_the_same_dataset(dataset_names[::-1], appended, min_sim=100)
# output_path = expanduser('~/workspace/dataset_merger/aggregated/{}_duplicates.csv'.format(dataset_names[1]))
# duplicates_df.to_csv(output_path, encoding='utf-8')
# duplicates_df

In [ ]:
%autoreload
dataset_names = ['wiki+googleart+wga+meisterwerke', 'moma']
sim = sim_matrix['-'.join(dataset_names)]
dfs_to_merge = [artists_with_years_dict[key] for key in dataset_names]

merged_df = get_merged_artists_df(dataset_names, dfs_to_merge, sim, 
                                  split_big_components=False, 
                                  allow_big_components=False) # TODO:
merged_df = merged_df[merged_df['works_count'] > 0]
artists_with_years_dict['+'.join(dataset_names)] = merged_df
print 'Num unique artists:', len(merged_df)
# merged_df.to_hdf('info/artists_{}_v0.93.hdf5'.format('+'.join(dataset_names)), 'df', mode='w')
merged_df.head()

In [ ]:
for key in artists_with_years_dict.keys():
    print '{}: {} unique artists'.format(key, len(artists_with_years_dict[key]))

In [ ]:
##### ARTUK #######
%autoreload
keys = ['wiki+googleart+wga+meisterwerke+moma', 'artuk']
# artists_with_years_dict['artuk'] = get_artists_with_years('artuk', dfs)
mask = (artists_with_years_dict['artuk']['years_bio'].notnull() |
        artists_with_years_dict['artuk']['wikidata_qid'].notnull() | 
        artists_with_years_dict['artuk']['artist_id_degruyter'].notnull() | 
        (artists_with_years_dict['artuk']['works_count'] > 6))
artists_with_years_dict['artuk'] = artists_with_years_dict['artuk'].loc[mask].copy()
print 'Num artists now:', len(artists_with_years_dict['artuk'])
get_sim_matrix_and_plot(keys, force=True, n_jobs=40, num_blocks=160)

In [ ]:
##### ARTUK #######
%autoreload
dataset_names = ['wiki+googleart+wga+meisterwerke+moma', 'artuk']
results_df = []
results_df = check_pairs(dataset_names, 
                transpose=False,
                min_sim=87, max_sim=121, 
                discard_exaclty_matched_dates=False,
                min_k=2,
                min_total_works_count=0)

if len(results_df):
    results_df['year_diff'] = results_df.apply(lambda x: [abs(x['dates_' + dataset_names[0]][i] - x['dates_' + dataset_names[1]][i]) for i in range(2)], axis=1)
results_df

# Transposed
results_df_transposed = []
results_df_transposed = check_pairs(dataset_names, 
                transpose=True,
                min_sim=87, max_sim=121, 
                discard_exaclty_matched_dates=False,
                min_k=2,
                min_total_works_count=0)
if len(results_df_transposed):
    results_df_transposed['year_diff'] = results_df_transposed.apply(lambda x: [abs(x['dates_' + dataset_names[0]][i] - x['dates_' + dataset_names[1]][i]) for i in range(2)], axis=1)

if len(results_df) and len(results_df_transposed):
    appended = results_df.append(results_df_transposed, ignore_index=True)
    appended = appended.drop_duplicates(['artist_id_' + key for key in dataset_names])
    appended = appended[results_df.columns]
#     assert len(appended['artist_id_meisterwerke'].unique()) == len(appended), len(appended['artist_id_meisterwerke'].unique())
    appended.sort_values(by='artist_id_' + dataset_names[0], inplace=True)
elif len(results_df):
    appended = results_df
elif len(results_df_transposed):
    appended = results_df_transposed
else:
    appended = pd.DataFrame()
    
# appended.sort_values(by='artist_id_wiki+googleart+wga', inplace=True)
# appended.sort_values(by='artist_id_meisterwerke', inplace=True)


from os.path import expanduser
output_path = expanduser('~/workspace/dataset_merger/aggregated/{}_check_matches.csv'.format('-'.join(dataset_names)))
appended.to_csv(output_path, encoding='utf-8')
print 'Total rows', len(appended)
print dataset_names
appended
# duplicates_df = get_pairs_from_the_same_dataset(dataset_names[::-1], appended, min_sim=100)
# output_path = expanduser('~/workspace/dataset_merger/aggregated/{}_duplicates.csv'.format(dataset_names[1]))
# duplicates_df.to_csv(output_path, encoding='utf-8')
# duplicates_df

In [ ]:
%autoreload
dataset_names = ['wiki+googleart+wga+meisterwerke+moma', 'artuk']
sim = sim_matrix['-'.join(dataset_names)]
dfs_to_merge = [artists_with_years_dict[key] for key in dataset_names]

merged_df = get_merged_artists_df(dataset_names, dfs_to_merge, sim, 
                                  split_big_components=False, 
                                  allow_big_components=True)
merged_df = merged_df[merged_df['works_count'] > 0]
artists_with_years_dict['+'.join(dataset_names)] = merged_df
print 'Num unique artists:', len(merged_df)
# merged_df.to_hdf('info/artists_{}_v0.93.hdf5'.format('+'.join(dataset_names)), 'df', mode='w')
merged_df.head()

In [ ]:
##### RIJKS #######
%autoreload
keys = ['wiki+googleart+wga+meisterwerke+moma+artuk', 'rijks']
dfs['rijks'] = dataset_merger.read_datasets.read_datasets(names=['rijks'])['rijks']
artists_with_years_dict['rijks'] = get_artists_with_years('rijks', dfs)
# mask = (artists_with_years_dict['rijks']['wikidata_qid'].notnull() | 
#         artists_with_years_dict['rijks']['artist_id_degruyter'].notnull() | 
#        (artists_with_years_dict['rijks']['works_count'] > 10))
# artists_with_years_dict['rijks'] = artists_with_years_dict['rijks'].loc[mask].copy()
print 'Num artists now:', len(artists_with_years_dict['rijks'])
get_sim_matrix_and_plot(keys, force=True, n_jobs=40, num_blocks=160)

In [ ]:
##### RIJKS #######
%autoreload
dataset_names = ['wiki+googleart+wga+meisterwerke+moma+artuk', 'rijks']
results_df = []
results_df = check_pairs(dataset_names, 
                transpose=False,
                min_sim=89, max_sim=121, 
                discard_exaclty_matched_dates=False,
                min_k=2,
                min_total_works_count=0)

if len(results_df):
    results_df['year_diff'] = results_df.apply(lambda x: [abs(x['dates_' + dataset_names[0]][i] - x['dates_' + dataset_names[1]][i]) for i in range(2)], axis=1)
results_df

# Transposed
results_df_transposed = []
results_df_transposed = check_pairs(dataset_names, 
                transpose=True,
                min_sim=89, max_sim=121, 
                discard_exaclty_matched_dates=False,
                min_k=2,
                min_total_works_count=0)
if len(results_df_transposed):
    results_df_transposed['year_diff'] = results_df_transposed.apply(lambda x: [abs(x['dates_' + dataset_names[0]][i] - x['dates_' + dataset_names[1]][i]) for i in range(2)], axis=1)

if len(results_df) and len(results_df_transposed):
    appended = results_df.append(results_df_transposed, ignore_index=True)
    appended = appended.drop_duplicates(['artist_id_' + key for key in dataset_names])
    appended = appended[results_df.columns]
#     assert len(appended['artist_id_meisterwerke'].unique()) == len(appended), len(appended['artist_id_meisterwerke'].unique())
    appended.sort_values(by='artist_id_' + dataset_names[0], inplace=True)
elif len(results_df):
    appended = results_df
elif len(results_df_transposed):
    appended = results_df_transposed
else:
    appended = pd.DataFrame()
    
# appended.sort_values(by='artist_id_wiki+googleart+wga', inplace=True)
# appended.sort_values(by='artist_id_meisterwerke', inplace=True)


from os.path import expanduser
output_path = expanduser('~/workspace/dataset_merger/aggregated/{}_check_matches.csv'.format('-'.join(dataset_names)))
appended.to_csv(output_path, encoding='utf-8')
print 'Total rows', len(appended)
print dataset_names
appended

In [ ]:
%autoreload
dataset_names = ['wiki+googleart+wga+meisterwerke+moma+artuk', 'rijks']
sim = sim_matrix['-'.join(dataset_names)]
dfs_to_merge = [artists_with_years_dict[key] for key in dataset_names]

merged_df = get_merged_artists_df(dataset_names, dfs_to_merge, sim, 
                                  split_big_components=False, 
                                  allow_big_components=True)
merged_df = merged_df[merged_df['works_count'] > 0]
artists_with_years_dict['+'.join(dataset_names)] = merged_df
print 'Num unique artists:', len(merged_df)
merged_df.to_hdf('info/artists_{}_v1.00.hdf5'.format('+'.join(dataset_names)), 'df', mode='w')
merged_df.head()

In [ ]:
# from fuzzywuzzy import fuzz
# def get_years_range_sim(range_a, range_b, is_bio=False, max_dist=85.0, max_delta=10,
#                         allow_include=True,
#                         allow_second_range_with_nan=False, bio_range=None):
#     """

#     Args:
#         range_a:
#         range_b:
#         is_bio:

#     Returns: similarity score from 0 to 1.0

#     """
#     if (np.isnan(range_a[0]) or np.isnan(range_b[0])) and \
#             (np.isnan(range_a[1]) or np.isnan(range_b[1])):
#         raise ValueError('Cannot compare 2 ranges with nans: {}, {}'.format(range_a, range_b))

#     if allow_second_range_with_nan:
#         range_a, range_b = list(range_a), list(range_b)
#         if np.isnan(range_b[0]):
#             assert not np.isnan(range_b[1]), 'one of the dates must be not none!'
#             range_b[0] = range_a[0]
#             if range_b[0] > range_b[1] + 3:
#                 return 0
#             max_dist = 0
#         elif np.isnan(range_b[1]):
#             assert not np.isnan(range_b[0]), 'one of the dates must be not none!'
#             range_b[1] = range_a[1]
#             if range_b[1] < range_b[0] - 3:
#                 return 0
#             max_dist = 0

#     assert isinstance(range_a, list) and isinstance(range_b, list)
#     ranges = [range_a, range_b]
#     ranges.sort()
#     for i in xrange(2):
#         ranges[i] = np.asarray(ranges[i], dtype=float)
#     assert np.all(np.isfinite(ranges)), ranges

#     if bio_range is not None:
#         max_dist = 0  # disable if we have bio
#         if allow_include and not np.isnan(bio_range[0]) and (ranges[0][0] < bio_range[0] - 1):
#             allow_include = False
#         if allow_include and not np.isnan(bio_range[1]) and (bio_range[1] + 1 < max(ranges[0][1], ranges[1][1])):
#             allow_include = False

#     if allow_include and ((ranges[0][0] <= ranges[1][0] and ranges[1][1] <= ranges[0][1]) or
#                           (ranges[1][0] <= ranges[0][0] and ranges[0][1] <= ranges[1][1])):
#         # one is subset of another
#         sim = 1.0
#     else:
#         dist = abs(ranges[1][1] - ranges[0][0])
#         delta = max(abs(ranges[1][1] - ranges[0][1]), abs(ranges[1][0] - ranges[0][0]))
#         assert dist >= 0, ranges
#         assert delta >= 0, ranges
#         # TODO: process if artist is still alive (death > 2017 ~2099)
#         # TODO: if one of the ranges is bio, allow only delta < 10
#         if dist <= max_dist or delta <= max_delta:
#             sim = 1.0
#         else:
#             if delta <= max_delta + 10:
#                 sim = 1 - (min((delta - max_delta), 10) / 10.0) ** 2
#             else:
#                 sim = 1 - np.sqrt(min((dist - max_dist), 35) / 35.0)
#     return sim


# def get_names_sim(names_a, names_b):
#     """

#     Args:
#         names_a:
#         names_b:

#     Returns: similarity score from 0 to 100

#     """
#     scores = list()
#     for a in names_a:
#         for b in names_b:
#             scores.append(fuzz.token_set_ratio(a, b, force_ascii=False))
#     return max(scores)


# def get_sim_urls_wiki(url_a, url_b):
#     urls = [url_a, url_b]
#     for i in xrange(len(urls)):
#         url = urllib.unquote(urls[i].encode('utf8')).decode('utf-8')
#         url = url.strip().lower()
#         tokens = url.split('wikipedia.org')
#         if len(tokens) != 2:
#             raise ValueError(u'Not a wiki link:{}'.format(url))
#         urls[i] = tokens[1]
#     return float(urls[0] == urls[1]) * 101


# def get_sim_between_rows(row_a, row_b):
#     names_a = row_a.artist_names
#     years_range_a = row_a.years_range
#     if hasattr(row_a, 'wikidata_qid') and hasattr(row_b, 'wikidata_qid') and \
#             not is_null_object(row_a.wikidata_qid) and not is_null_object(row_b.wikidata_qid):
#         cur_sim = (row_a.wikidata_qid.strip().lower() == row_b.wikidata_qid.strip().lower()) * 101
#     elif hasattr(row_a, 'url_wiki') and hasattr(row_b, 'url_wiki') and \
#          not is_null_object(row_a.url_wiki) and not is_null_object(row_b.url_wiki) and \
#          'wikipedia.org' in row_a.url_wiki and \
#          'wikipedia.org' in row_b.url_wiki:
#         cur_sim = get_sim_urls_wiki(row_a.url_wiki, row_b.url_wiki)
#     elif hasattr(row_a, 'artist_id_degruyter') and hasattr(row_b, 'artist_id_degruyter') and \
#          not is_null_object(row_a.artist_id_degruyter) and not is_null_object(row_b.artist_id_degruyter):
#         cur_sim = (row_a.artist_id_degruyter == row_b.artist_id_degruyter) * 101
#     else:
#         print 'General sim'
#         names_b = row_b.artist_names
#         years_range_b = row_b.years_range
#         years_sim = get_years_range_sim(years_range_a, years_range_b, max_dist=-1)
#         if years_sim < 0.5:
#             cur_sim = years_sim * 0.9
#         else:
#             names_sim = get_names_sim(names_a, names_b)
#             print 'names_sim', names_sim, 'years_sim', years_sim
#             cur_sim = years_sim * names_sim
#     return cur_sim


# for item in appended.itertuples():
#     idx = item.idx[::-1]
# #     print map(len, [artists_with_years_dict[x] for x in dataset_names])
#     row_a = list(artists_with_years_dict[dataset_names[0]].iloc[[idx[0]]].itertuples())[0]
#     row_b = list(artists_with_years_dict[dataset_names[1]].iloc[[idx[1]]].itertuples())[0]
    
#     new_sim = get_sim_between_rows(row_a, row_b)
#     print '{}->{}'.format(sim_matrix['wiki+googleart+wga+meisterwerke+moma+artuk-rijks'][idx[0], idx[1]], 
#                           new_sim)
# #     sim_matrix['wiki+googleart+wga+meisterwerke+moma+artuk-rijks'][idx[0], idx[1]] = new_sim

In [ ]:
z[z.index.str.startswith('artuk_austin, samuel,')]

In [ ]:
z[z.index.str.startswith('artuk_colkett, samuel david,')]

In [ ]:
z = artists_with_years_dict['rijks']
z[z.index.str.startswith('rijks_rjgd-suzuki-kiitsu')].values

In [ ]:
z = artists_with_years_dict['wiki+googleart+wga+meisterwerke+moma+artuk']
z[z.index.str.startswith('artuk_denning, stephen poyntz, (1795-1864)')]
